<a href="https://colab.research.google.com/github/noailabs/AI-agents-hacks/blob/main/codegen_colab_openrouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os
os.environ["OPENROUTER_API_KEY"]=userdata.get('OPENROUTER_API_KEY')

In [ ]:
%%writefile app.py
from openai import OpenAI
import os
import json

model="deepseek/deepseek-chat-v3-0324:free"
#model="meta-llama/llama-4-maverick:free"

settings={
    "temperature":1.0,
    "top_p":1.0
}

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.environ["OPENROUTER_API_KEY"],
)

SYSTEM_PROMPT="""ONLY USE HTML, CSS AND JAVASCRIPT.
If you want to use ICON make sure to import the library first.
Try to create the best UI possible by using only HTML, CSS and JAVASCRIPT.
Use as much as you can TailwindCSS for the CSS,
if you can't do something with TailwindCSS,
then use custom CSS (make sure to import <script src="https://cdn.tailwindcss.com"></script> in the head).
Also, try to ellaborate as much as you can, to create something unique.
ALWAYS GIVE THE RESPONSE INTO A SINGLE HTML FILE.
"""

prompt="""Generate a web canvas with a wireframe star
"""

extra="""
"""

messages=[]

try:
    file_path="messages.json"
    with open(file_path, "r") as json_file:
        messages = json.load(json_file)

    print("\nLoaded messages:")
    for msg in messages:
        print(f"{msg['role']}: {msg['content']}")
    print("-----")
except FileNotFoundError:
    print(f"Warning: {file_path} not found.")
except json.JSONDecodeError:
    print(f"Error: Invalid JSON in {file_path}.")
    sys.exit(0)

messages.append({"role": "user","content": prompt})

prompt=prompt+extra

completion = client.chat.completions.create(
#  extra_headers={
#    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
#    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
#  },
  #extra_body={},
  model=model,
  messages=[
    {
      "role": "system",
      "content": SYSTEM_PROMPT
    },
    {
      "role": "user",
      "content": prompt
    }
  ],
  **settings
)

print(completion.choices[0].message.content)

Writing app.py


In [ ]:
!python app.py

Traceback (most recent call last):
  File "/content/app.py", line 36, in <module>
    completion = client.chat.completions.create(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/resources/chat/completions/completions.py", line 914, in create
    return self._post(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1242, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 919, in request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.p

In [ ]:
%%writefile app.py
#from huggingface_hub import InferenceClient
from openai import OpenAI
import json
import sys
import argparse
import os

def load_env_file(filepath='.env'):
    """
    Load variables from a .env file and return them as a dictionary.

    Args:
        filepath (str): Path to the .env file. Defaults to '.env' in current directory.

    Returns:
        dict: Dictionary containing the key-value pairs from the .env file
    """
    env_vars = {}

    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                # Skip empty lines and comments
                if not line or line.startswith('#'):
                    continue

                # Split on first occurrence of '='
                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip()

                    # Remove surrounding quotes if present
                    if len(value) > 1 and (
                        (value.startswith('"') and value.endswith('"')) or
                        (value.startswith("'") and value.endswith("'"))
                    ):
                        value = value[1:-1]

                    env_vars[key] = value
    except FileNotFoundError:
        print(f"Warning: {filepath} not found.")
    except Exception as e:
        print(f"Error loading .env file: {e}")

    return env_vars

#env = load_env_file()
#token=env["HF_TOKEN"]
#OPENROUTER_API_KEY=env["OPENROUTER_API_KEY"]
OPENROUTER_API_KEY=os.environ["OPENROUTER_API_KEY"]

# print(token)

#model="deepseek-ai/DeepSeek-V3-0324"
model="deepseek/deepseek-chat-v3-0324:free"

provider="sambanova"

settings={
    "temperature":1.0,
    "top_p":1.0,
}

stream=True





parser = argparse.ArgumentParser(description="Process a prompt for an AI model.")

parser.add_argument(
        "-p", "--prompt",
        type=str,
        required=True,
        help="The prompt to send to the AI model"
)

parser.add_argument(
        "-o", "--output",
        type=str,
        required=False,
        help="The output file"
)


args = parser.parse_args()


prompt=args.prompt

output_file=args.output

# prompt="tell a joke about a corner"

SYSTEM_PROMPT="""You are code generation assistant.
Don't add any comments before or after code, you can put it inside as comments blocks.
Don't start or end with ```, just generate code.
"""



messages=[{"role":"system","content":SYSTEM_PROMPT}]

try:
    file_path="messages.json"
    with open(file_path, "r") as json_file:
        messages = json.load(json_file)

    print("\nLoaded messages:")
    for msg in messages:
        print(f"{msg['role']}: {msg['content']}")
    print("-----")
except FileNotFoundError:
    print(f"Warning: {file_path} not found.")
except json.JSONDecodeError:
    print(f"Error: Invalid JSON in {file_path}.")
    sys.exit(0)

messages.append({"role": "user","content": prompt})


client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)


#client = InferenceClient(
#    token=token,
#    provider=provider, #"hyperbolic", #"together",
#    model=model,
#    #api_key="my_together_api_key",
#)
#resp=client.chat_completion(messages, max_tokens=1000)



response=""
if stream:
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True,
    **settings,
  )
  for chunk in completion:
    content = chunk.choices[0].delta.content
    if content is not None:
        response+=content
        sys.stdout.write(content)
        sys.stdout.flush()
  print()
else:
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=False,
    **settings,
  )
  response=completion.choices[0].message.content
  print(response)

messages.append({"role": "assistant", "content": response})



if output_file!=None:
  with open('output_file', 'w') as file:
     start_idx=response.find("```", 0)

    file.write(code)

with open("messages.json", "w") as json_file:
    json.dump(messages, json_file, indent=4)

Overwriting app.py


In [ ]:
!rm messages.json

rm: cannot remove 'messages.json': No such file or directory


In [ ]:
!python app.py -p "Generate 2048 game clone web app,output single html file"


Loaded messages:
system: You are code generation assistant.
Don't add any comments before or after code, you can put it inside as comments blocks.
Don't start or end with ```, just generate code.

user: Generate treejs rotating cube web app, output single html file
assistant: <!DOCTYPE html>
<html>
<head>
    <title>Rotating Cube</title>
    <style>
        body { margin: 0; overflow: hidden; }
        canvas { display: block; }
    </style>
</head>
<body>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>
        // Scene setup
        const scene = new THREE.Scene();
        const camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
        const renderer = new THREE.WebGLRenderer({ antialias: true });
        
        renderer.setSize(window.innerWidth, window.innerHeight);
        document.body.appendChild(renderer.domElement);
        
        // Create cube
        const geometry = ne

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr

# Your HTML content as a string
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>My HTML</title>
    <style>
        body { font-family: Arial; margin: 20px; }
        .header { color: #3366ff; }
    </style>
</head>
<body>
    <h1 class="header">Welcome to my HTML content</h1>
    <p>This HTML is rendered directly from a Python string.</p>
    <ul>
        <li>Item 1</li>
        <li>Item 2</li>
        <li>Item 3</li>
    </ul>
</body>
</html>
"""

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## HTML Content Display")

    # Display the HTML content
    gr.HTML(html_content)

    # Optional: Add controls to modify the HTML
    with gr.Accordion("Edit HTML", open=False):
        html_editor = gr.Textbox(value=html_content, lines=10, label="HTML Code")
        update_btn = gr.Button("Update")

        def update_html(new_html):
            return new_html

        update_btn.click(
            fn=update_html,
            inputs=html_editor,
            outputs=gr.HTML()
        )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5042ccbaa33913670c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
s1="""
Code for
```python
#asdasd
#asdasd

a=```

holy cow
```

"""
s1=s1.strip()
mark="```"
p=s1.find(mark)
if p != -1:
    print(f"Found ``` index {p}")
    s1=s1[p+len(mark):]
    p = s1.find("\n")
    if p == -1:
      print("Something wrong!!!")
    p=s1.rfind(mark)
    s1=s1[:p-1]
print(s1)


a="""
while True:
  p = s1.find("\n")
  if p != -1:
      print(f"Newline found at index {p}")
      s1=s1[p+1:]
  else:
      print("No newline found")
      break
"""

Found ``` index 9
python
#asdasd
#asdasd

a=```

holy cow
